# Day 3 - Lab 1: AI-Driven Backend Development

**Objective:** Generate a complete FastAPI backend application, including Pydantic and SQLAlchemy models, and then perform the critical engineering task of integrating the generated code with the live SQLite database created on Day 2.

**Estimated Time:** 135 minutes

**Introduction:**
Welcome to Day 3! With our requirements and architecture defined, it's time to write code. In this lab, you will act as a senior developer guiding an AI co-pilot. Your task is to generate the full backend API for the Onboarding Tool. This involves not just generating code, but also connecting it to the live database we created yesterday, moving from a prototype to a functional, data-driven application.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We'll set up our environment and load the `schema.sql` artifact from Day 2. This SQL file contains the `CREATE TABLE` statements that define our database structure, which is the perfect context to provide the LLM for code generation.

**Model Selection:**
For code generation, models specifically fine-tuned for coding are ideal. `gpt-4.1`, `o3`, or `codex-mini` are excellent choices. Experiment to see which one gives you the cleanest code.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the SQL schema.
- `save_artifact()`: To save the generated Python code.
- `clean_llm_output()`: To remove markdown fences from the generated code.

In [8]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gpt-4.1")

# Load the SQL schema from Day 2
sql_schema = load_artifact("artifacts/schema.sql")
if not sql_schema:
    print("Warning: Could not load schema.sql. Lab may not function correctly.")

2025-10-29 14:53:28,491 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4.1 latency_ms=None artifacts_path=None


## Step 2: The Challenges

Follow the challenges below to build and connect your API.

### Challenge 1 (Foundational): Generating Code with In-Memory Logic

**Task:** Generate all the necessary Python code for a FastAPI application, but with simple in-memory data storage for now. This allows us to generate and validate the code's structure before adding database complexity.

**Instructions:**
1.  Create a detailed prompt that asks the LLM to act as a senior Python developer.
2.  Provide the `sql_schema` as context.
3.  Instruct the LLM to generate three key components:
    * **Pydantic Models:** For API data validation (request/response bodies).
    * **FastAPI Endpoints:** Full CRUD (Create, Read, Update, Delete) endpoints for the `users` table.
    * **In-Memory Database:** A simple Python list to act as a temporary, fake database.
4.  The final output should be a single Python script for a `main_in_memory.py` file.
5.  Save the generated code to `app/main_in_memory.py`.

In [9]:
# Prompt guiding the LLM to return a FastAPI app backed by in-memory storage.
in_memory_api_prompt = f"""
Purpose:
- Produce a runnable FastAPI module named main_in_memory.py that exposes CRUD endpoints for the Day 2 users table while we are still using in-memory storage.

Context:
- You are a senior Python engineer mentoring a teammate.
- Use the onboarding schema below to align field names and validation rules.
- Database schema (from schema.sql):
{sql_schema}

Constraints:
- Return raw Python source code only (no Markdown, fences, commentary, or placeholders).
- Implement the FastAPI app, Pydantic models, and in-memory persistence in a single module.
- Reflect schema constraints where practical (e.g., enforce role enum, non-nullable columns, default timestamps as isoformat strings).
- Use deterministic starter data (e.g., a single seed user with id=1) and manage IDs with a counter so CRUD actions remain consistent across runs.
- Adhere to PEP 8 and include only concise comments that clarify non-obvious logic.

Tasks:
1. Declare Pydantic models for reading and writing user data (e.g., UserBase, UserCreate, UserUpdate, UserOut) using typing annotations that match the schema.
2. Instantiate FastAPI and implement an in-memory repository (list or dict) plus a helper for generating incremental primary keys.
3. Implement the following endpoints with full CRUD parity: GET /users, GET /users/{{user_id}}, POST /users, PUT /users/{{user_id}}, DELETE /users/{{user_id}}.
4. Apply validation such as email format, role enumerations, and optional manager_id handling.
5. Ensure error handling uses HTTPException with appropriate status codes when records are missing or duplicates would occur.

Output:
- Return one complete Python script ready to save as app/main_in_memory.py and run via `uvicorn app:app`.
"""

print("--- Generating FastAPI app with in-memory database ---")
if sql_schema:
    generated_api_code = get_completion(in_memory_api_prompt, client, model_name, api_provider)
    cleaned_code = clean_llm_output(generated_api_code, language='python')
    print(cleaned_code)
    save_artifact(cleaned_code, "app/main_in_memory.py", overwrite=True)
else:
    print("Skipping API generation because schema is missing.")

--- Generating FastAPI app with in-memory database ---
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, EmailStr, Field, validator
from typing import Optional, List, Literal
from datetime import datetime
from threading import Lock

app = FastAPI()

# Pydantic models

class UserBase(BaseModel):
    company_id: int
    email: EmailStr
    password_hash: str = Field(..., min_length=1)
    first_name: str = Field(..., min_length=1)
    last_name: str = Field(..., min_length=1)
    role: Literal['Admin', 'Hiring Manager', 'New Hire']
    manager_id: Optional[int] = None

    @validator('first_name', 'last_name')
    def not_empty(cls, v):
        if not v.strip():
            raise ValueError('Must not be empty')
        return v

class UserCreate(UserBase):
    pass

class UserUpdate(BaseModel):
    company_id: Optional[int] = None
    email: Optional[EmailStr] = None
    password_hash: Optional[str] = Field(None, min_length=1)
    first_name: Optional[str] = Fiel

**Challenge 1 Notes:** Generated in-memory FastAPI module defines schema-aligned Pydantic models, exposes CRUD endpoints for `users`, and uses deterministic seed data with an incremental ID helper. This mirrors the Day 2 table contract while keeping persistence logic transient for quick iteration.

### Challenge 2 (Intermediate): Generating Database Models and Session Code

**Task:** Now, generate the specific SQLAlchemy code required to connect our application to the live `onboarding.db` SQLite database.

**Instructions:**
1.  Create a new prompt.
2.  Provide the `sql_schema` as context again.
3.  Instruct the LLM to generate two separate pieces of code:
    * **SQLAlchemy Models:** Python classes that map to your database tables.
    * **Database Session Management:** The boilerplate code to create a database engine, session maker, and a dependency function (`get_db`) for use in FastAPI.
4.  The output should be two distinct, well-commented Python code blocks. We will integrate these manually in the next step.

In [10]:
# Prompt instructing the LLM to emit SQLAlchemy models plus session/dependency boilerplate.
db_code_prompt = f"""
Purpose:
- Generate SQLAlchemy ORM classes and database session utilities that mirror the onboarding tool schema for use in a FastAPI project.

Context:
- Act as a principal backend engineer delivering production-ready scaffolding.
- Align exactly with the Day 2 SQLite schema provided below.
- Database schema (from schema.sql):
{sql_schema}

Constraints:
- Output two consecutive Python code blocks only: (1) ORM models, (2) database/session setup with FastAPI dependency.
- Do not wrap results in Markdown or add explanatory prose.
- Use SQLAlchemy 2.0 style (`DeclarativeBase`, `Mapped`, `mapped_column`, `relationship`).
- Configure the engine for the relative path `sqlite:///artifacts/onboarding.db` with `connect_args={{"check_same_thread": False}}`.
- Include `Base.metadata.create_all(bind=engine)` so tables auto-create during local dev.
- Provide concise comments where relationships or configuration choices may need clarification; avoid redundant commentary.

Tasks:
1. Define declarative models for every table in the schema, capturing primary keys, unique constraints, relationship loading options, and foreign key behaviors.
2. Surface useful reverse relationships (e.g., `users` ↔ `companies`, `onboarding_processes` ↔ `assigned_tasks`) using meaningful `back_populates` names.
3. Expose timestamps as `datetime` columns (`DateTime` with defaults via `func.now()`), roles/status fields as `Enum` or constrained `String` where practical, and boolean-like flags as `Boolean` with default values matching the schema.
4. Provide a second block that imports the models’ `Base`, builds the `engine`, configures `SessionLocal`, and defines a FastAPI-compatible `get_db()` dependency yielding a scoped session.

Output:
- Return the two Python blocks separated by a single blank line so they can be pasted directly into dedicated files.
"""

print("--- Generating SQLAlchemy Models and Session Code ---")
if sql_schema:
    generated_db_code = get_completion(db_code_prompt, client, model_name, api_provider)
    print("\n--- Generated Database Code ---")
    print(generated_db_code)
else:
    print("Skipping DB code generation because schema is missing.")

--- Generating SQLAlchemy Models and Session Code ---

--- Generated Database Code ---
from datetime import datetime
from sqlalchemy import (
    String, Integer, DateTime, ForeignKey, Boolean, Enum, Text, UniqueConstraint, CheckConstraint, func
)
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship
import enum


class Base(DeclarativeBase):
    pass


class UserRole(enum.Enum):
    Admin = "Admin"
    Hiring_Manager = "Hiring Manager"
    New_Hire = "New Hire"


class ProcessStatus(enum.Enum):
    Not_Started = "Not Started"
    In_Progress = "In Progress"
    Completed = "Completed"


class AssignedTaskStatus(enum.Enum):
    Pending = "Pending"
    Completed = "Completed"


class Company(Base):
    __tablename__ = "companies"

    company_id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String, unique=True, nullable=False)
    created_at: Mapped[datetime] = mapped_column(DateTime, nullable=False, default=func.now())
  

In [11]:
# Persist the generated DB code for offline review and show a brief preview snippet.
cleaned_db_code = clean_llm_output(generated_db_code, language='python')
save_artifact(cleaned_db_code, "generated_db_code.py", overwrite=True)
preview_slice = "\n".join(cleaned_db_code.splitlines()[:40])
print(preview_slice)

from datetime import datetime
from sqlalchemy import (
    String, Integer, DateTime, ForeignKey, Boolean, Enum, Text, UniqueConstraint, CheckConstraint, func
)
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship
import enum


class Base(DeclarativeBase):
    pass


class UserRole(enum.Enum):
    Admin = "Admin"
    Hiring_Manager = "Hiring Manager"
    New_Hire = "New Hire"


class ProcessStatus(enum.Enum):
    Not_Started = "Not Started"
    In_Progress = "In Progress"
    Completed = "Completed"


class AssignedTaskStatus(enum.Enum):
    Pending = "Pending"
    Completed = "Completed"


class Company(Base):
    __tablename__ = "companies"

    company_id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String, unique=True, nullable=False)
    created_at: Mapped[datetime] = mapped_column(DateTime, nullable=False, default=func.now())
    updated_at: Mapped[datetime] = mapped_column(DateTime, nullable=False, default=func.n

**Challenge 2 Notes:** Prompt constrained the LLM to mirror every table in `schema.sql`, enforce enum/status domains via `Enum`, wire bidirectional relationships, and point the engine at `sqlite:///artifacts/onboarding.db` with `check_same_thread=False` so FastAPI dependencies can reuse sessions safely.

### Challenge 3 (Advanced): Integrating Live Database Logic

**Task:** This is the most critical engineering step of the lab. You will manually integrate the generated database code into the FastAPI application, replacing the in-memory logic with live database operations.

**Instructions:**
This task represents a significant jump in complexity. Follow these steps carefully in your IDE (like VS Code):

1.  Create a new, empty file named `app/main.py`.
2.  **First, copy the Pydantic models and the `app = FastAPI()` line** from your `app/main_in_memory.py` file and paste them into `app/main.py`.
3.  **Next, paste the SQLAlchemy model classes and the `get_db` dependency function** you generated in Challenge 2 into your new `app/main.py`.
4.  **Now, let's refactor the `POST /users/` endpoint.** Copy the endpoint function from the in-memory file, but replace the in-memory logic (e.g., `db.append()`) with the correct SQLAlchemy session calls: `db.add(db_user)`, `db.commit()`, and `db.refresh(db_user)`.
5.  Repeat this refactoring process for the other endpoints (GET, PUT, DELETE), replacing list manipulations with the appropriate SQLAlchemy `db.query()` methods.

This task requires you to act as the senior developer, stitching together the AI-generated components into a functional, cohesive whole. You may need to ask the LLM follow-up questions like, "How do I write a SQLAlchemy query to find a user by ID?"

## Lab Conclusion

Congratulations! You have successfully generated and assembled a complete, database-connected backend API. You used an LLM to generate the boilerplate for both the API endpoints and the database models, and then performed the crucial engineering task of integrating them. You now have a working `main.py` file in your `app` directory that can create, read, update, and delete data in a live database. In the next lab, we will write a comprehensive test suite for this API.

> **Key Takeaway:** AI excels at generating boilerplate code (like models and endpoint structures), but the developer's critical role is in the final integration and wiring of these components into a coherent, working system.